# Demo the Liquidator Agent Logic

This notebook:

1. Fetches the latest crvusd contract data into Python objects using `crvusdsim`.
2. Fetches prices and `ExternalMarket`s from the `baseline` scenario.
3. Instantiates a liquidator agent.
4. Checks if there are users to liquidate in the `Controller` and liquidates them.

TODO describe the liquidation logic in detail.

In [1]:
from src.sim.scenario import Scenario

In [2]:
# Generate markets and prices
scenario = Scenario("baseline")
sample = scenario.pricepaths[0]
scenario.prepare_for_run()  # Set block timestamps
scenario.prepare_for_trades(sample)  # Set External Market Price

# Unpack
markets = scenario.markets
liquidator = scenario.liquidator
controller = scenario.controller

[DEBUG][16:56:48][root]-762369: Using 614880 1Inch quotes.
[INFO][16:56:48][root]-762369: Fetching sim_market from subgraph.
[INFO][16:56:51][root]-762369: Found 20 valid cycles of length 3.


In [3]:
# Artificially inflate collateral price
collateral = scenario.controller.COLLATERAL_TOKEN.address
sample = scenario.pricepaths[0]
prices_usd = sample.prices_usd
assert collateral in prices_usd.keys()
prices_usd[collateral] *= 2
sample.update(prices_usd)
scenario.update_market_prices(sample)

In [4]:
profit, count = liquidator.perform_liquidations(controller)
print(f"Liquidated {count} positions for a profit of {profit} USD.")

[INFO][16:56:51][root]-762369: There are 54 users to liquidate.
[INFO][16:56:51][root]-762369: Missed liquidation for user 0xf165d7273288ace1e7a317d6d23956e46fe229c5. Health: -739277275637517056.000000. Expected profit: -11400.016216.
[INFO][16:56:51][root]-762369: Missed liquidation for user 0xf11f0add0e8e4ee208104d8264fcf1b69c4ceafc. Health: -739897610837227392.000000. Expected profit: -13351.255919.
[INFO][16:56:51][root]-762369: Missed liquidation for user 0xec718904654b29f1f571290259c324fd2e3aa63e. Health: -880329190498585472.000000. Expected profit: -42612.278274.
[INFO][16:56:51][root]-762369: Missed liquidation for user 0xebdeff4d7053bf84262d2f9fc261a900c4323d83. Health: -661471498696840064.000000. Expected profit: -4173.710057.
[INFO][16:56:51][root]-762369: Liquidating user 0xea3f9b017c6b811b0a8ca642346bd805d936fce4 with expected profit: 2899.485315.
[INFO][16:56:51][root]-762369: Executing cycle Cycle(Trades: [Swap(pool=Curve.fi Factory Plain Pool: crvUSD/USDC, in=USDC, out=

Liquidated 15 positions for a profit of 992566.46633 USD.
